# Distributed Transactions

# Concepts
* [分布式事务](http://icyfenix.cn/architect-perspective/general-architecture/transaction/distributed.html)


## CAP
* [wikipedia](https://en.wikipedia.org/wiki/CAP_theorem)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/CAP_Theorem_Venn_Diagram.png/1024px-CAP_Theorem_Venn_Diagram.png" width="400"/>

CAP猜想: 一个分布式的系统中，涉及**共享数据**问题时，以下三个特性最多只能同时满足其中两个：
* 一致性(Consistency)：代表数据在任何时刻、任何分布式节点中所看到的都是符合预期的。
* 可用性(Availability)：代表系统不间断地提供服务的能力，理解可用性要先理解与其密切相关两个指标：可靠性(Reliability)和可维护性(Serviceability)。
  * 可靠性使用平均无故障时间(Mean Time Between Failure，MTBF)来度量；
  * 可维护性使用平均可修复时间(Mean Time To Repair，MTTR)来度量。
  * 可用性衡量系统可以正常使用的时间与总时间之比，其表征为：A=MTBF/(MTBF+MTTR)，即可用性是由可靠性和可维护性计算得出的比例值，譬如 99.9999%可用，即代表平均年故障修复时间为 32 秒。
* 分区容忍性(Partition Tolerance)：代表分布式环境中部分节点因网络原因而彼此失联后，即与其他节点形成“网络分区”时，系统仍能正确地提供服务的能力。

tradeoff:
* CA: 分布式环境中分区是永远可能存在的.
* CP: 不要求可用性, 要求一致性, 即数据完全同步, 在分区时导致同步时间过长.
* AP: 不要求一致性, 要求可用性, 存在数据不一致问题.

application:
* ZooKeeper: CP
  * 任何读请求得到的是一致性的结果.
  * 在Leader选举过程中或半数以上机器不可用时, 服务是不可用的.
* Eureka: AP
  * 设计时保证可用性: 只要有一个节点可用就提供服务, 但不能保证节点上的数据是最新的.
* Nacos: 
  * AP(默认): 客户端注册时ephemeral=true, 由distro协议实现.
  * CP: 客户端注册时ephemeral=false, 由raft协议实现.

## BASE
* [Base: An Acid Alternative](https://queue.acm.org/detail.cfm?id=1394128): Dan Pritchett, Ebay, 2008

BASE理论指的是
* Basically Available: 基本可用
  * 允许损失部分可用性, 例如响应时间上的损失, 功能上的损失(降级).
* Soft state: 软状态
  * 允许系统中的数据存在不影响系统整体可用性的中间状态, 例如允许副本之间同步存在延时.
* Eventually consistent: 最终一致性
  * 系统中所有副本在一段时间的同步后, 最终能够达到一致的状态.

BASE理论是对CAP中的*一致性和可用性*进行一个*权衡*的结果，理论的核心思想就是：我们*无法做到强一致*，但每个应用都可以根据自身的业务特点，采用适当的方式来使系统*达到最终一致性*(Eventual consistency)。


如何保证最终一致性:
* 读时修复: 读取数据时检测是否一致, 不一致时进行修复. 例: Cassandra的Read Repair.
* 写时修复: 写入数据时检测是否一致, 不一致时进行修复. 例: Cassandra的Hinted Handoff, 远程写失败时将数据缓存, 之后定时重传. 
* 异步修复: 使用系统外的机制检测数据一致性, 并修复. 例: 定时对账.

# Solutions

* [ref](https://cloud.tencent.com/developer/article/1806989)

## 2PC

角色:
* 事务协调者: A
* 事务参与者: B, C

第一个阶段: 投票阶段/准备阶段
* 事务协调者A将命令写入日志.
* 事务协调者A发送prepare命令给B和C.
* B和C收到prepare命令后, 判断自己实际是否可以提交, 并将结果写入日志.
* B和C将结果返回给事务协调者A.

第二个阶段: 决定阶段
* 事务协调者A收到B和C的prepare命令响应后, 判断事务是否可以提交
  * 如果可以, 则写入日志, 发送commit命令
  * 如果不可以, 则加入日志, 发送abort命令
* 事务参与者B和C收到commit/abort命令后, 执行命令, 并将命令和命令执行结果写入日志
* 事务参与者B和C将命令执行结果返回给事务协调者A.

存在问题:
* 单点故障: 事务管理器出现故障, 系统不可用.
* 数据不一致: 在第二阶段, 如果事务协调者只发送了部分commit/abort命令, 只有部分参与者提交/回滚, 系统数据存在不一致.
* 响应时间长: 整个消息链路是串行的.
* 不确定性: 事务协调者判断完事务是否可以提交后失败, 系统处于不确定状态.

## 3PC

2PC + CanCommit阶段 + 超时机制
* 解决了单点故障: 参与者在一段时间内未收到协调者的提交/回滚命令, 则中断事务执行.
* 未解决性能和不一致问题

CanCommit阶段:
* 事务协调者询问参与者, 是否有能力完成事务
  * 都返回yes, 进入PreCommit阶段;
  * 有一个返回no或等待响应超时, 中断事务, 向所有参与者发送abort命令.

PreCommit阶段:
* 事务协调者向所有参与者发送PreCommit命令, 参与者收到后开始执行事务中操作, 并将Undo和Redo信息记录到事务日志中, 向协调者反馈Ack, 并等待后续命令.

DoCommit阶段:
* 事务协调者等待所有协调者的反馈
  * 收到所有反馈: 向所有参与者发送DoCommit命令, 参与者收到后各自执行事务提交操作, 向协调者反馈Ack, 协调者完成事务.
  * 如果一个参与者的反馈未完成PreCommit或反馈超时, 协调者向所有参与者发送abort命令, 以中断事务.

## TCC

补偿机制: 对每个操作, 都关联相应的确认和补偿撤销操作.

Try阶段: 执行业务系统检测和资源预留
* Confirm阶段: 对业务系统执行确认提交. 默认是不会出错的.
* Cancel阶段: 业务执行出错时, 撤销业务操作, 并释放预留资源.

例: 支付系统, 库存系统

优缺点:
* 解决了协调者单点问题: 主业务发起方充当协调者.
* 解决了同步阻塞问题: 引入超时, 超时后补偿.
* 保证数据一致性: 主业务发起方通过补偿操作保证.
* 增加了业务代码的复杂性.

## 本地消息表

消息表: 记录消息发送状态, 与业务数据在同一个事务中操作.

业务发起方:
* 在一个事务中提交消息记录和业务数据变更: 标记消息为发送中.
* 通过消息队列发送消息, 并将消息标记为已发送.
* 如果消息发送失败, 重试发送: 补偿失败消息, 最大重试次数.

下游业务:
* 处理消息, 完成业务逻辑: 幂等处理.
* 如果处理业务逻辑失败, 给业务发起方发送一个业务补偿消息, 通知其回滚.
* 如果处理业务逻辑成功, 给业务发起方发送一个消息消费成功的消息: 业务发起方标记消息为已处理.

## 消息事务

将本地消息表的操作委托给消息中间件: 例RocketMQ

流程:
* 业务发起方发送prepare消息到消息中间件
* 发送成功后, 执行本地事务
  * 成功: 执行commit, 消息中间件将消息下发给客户端.
  * 失败: 执行abort, 消息中间件将prepare消息删除.
* 下游业务: 幂等消费消息. 

## 最大努力通知

对一致性要求较低的场景.

流程:
* 上游业务本地事务执行完后, 发送消息到消息队列.
* 专用的消息消费服务执行消息消费, 并调用下游业务接口.
  * 成功: 完成.
  * 失败: 在最大重试次数约束下重试调用下游业务接口.

## Saga

长时间运行的事务: 将长事务拆分为多个本地短事务, 由事务协调器负责协调, 如果正常结束则完成, 如果某个短事务失败, 则根据相反的顺序调用补偿操作.

例: Seata SAGA.

# Apache Seata
* https://seata.apache.org/
* code: https://github.com/apache/incubator-seata
* example: https://github.com/apache/incubator-seata-samples

> What is Seata?
>
> Simple Extensible Autonomous Transaction Architecture, 简单可扩展的匿名事务架构
>
> A distributed transaction solution with high performance and ease of use for microservices architecture.

actions:
* https://github.com/jargonzhou/application-store/tree/main/ops/seata
* workbench/Java/JavaEE/example-springcloud/persistent/seata
* HTTPie: Archived > Seata

领域模型:
* TC: Transaction Coordinator, 事务协调者 - Seata Server
  * 维护全局和分支事务的状态, 驱动全局事务的提交或回滚.
* TM: Transaction Manager, 事务管理器 - 业务发起方
  * 定义全局事务的范围: 包括开启, 提交或回滚.
* RM: Resource Manager, 资源管理器 - 下游业务
  * 管理分支事务处理的资源, 与TC交互以注册分支事务和报告分支事务状态, 并驱动分支事务的提交或回滚.

事务模式: [link](https://seata.apache.org/zh-cn/docs/dev/mode/at-mode), [talk](分布式事务 Seata 及其三种模式详解
)
- AT: 无业务入侵的即插即用
  - 前提: 支持本地ACID事务的关系型数据库, JDBC应用.
- TCC: 不与服务框架耦合, 与底层RPC协议无关, 与底层存储介质无关
- SAGA: 高度自定义, 最终一致性, 高性能
- XA: 无业务入侵的即插即用
  - 前提: 支持XA事务的数据库, JDBC应用.

AT: 使用数据源代理DataSourceProxy, 涉及的逻辑包括插入回滚undo_log日志, 检查全局锁等.

一阶段:
* 业务数据和回滚日志记录在同一个本地事务中提交, 释放本地锁和连接资源.
二阶段:
* 提交异步化
* 回滚通过一阶段中的回滚日志进行反向补偿.

写隔离:
* 一阶段本地事务提交前, 需要确保先拿到全局锁. 拿不到全局锁, 不能提交本地事务.
* 拿全局锁的尝试: 限制在一定范围内, 超出则放弃, 回滚本地事务并释放本地锁.

读隔离
* 本地数据库各级级别读已提交RC或以上, AT模式默认全局隔离级别是读未提交RU.
* 必须要求全局的读已提交RC, 使用`SELECT FOR UPDATE`.

流程:
* TM向TC申请开启一个全局事务XID.
* XID在业务调用链路上传播.
* RM开始执行分支事务, 解析SQL生成UNDO_LOG记录(XID, Branch ID, beforeImage, afterImage数据).
* RM在同一个本地事务中**执行**业务SQL和插入UNDO_LOG记录
* RM向TC**注册**分支事务, 并**申请业务记录的全局锁**,
* RM本地**提交**分支事务, 并向TC报告本地事务执行成功.
* TC根据所有分支事务执行结果, 向RM下发commit/abort命令.
  * RM收到commit: **释放业务记录全局锁**, 将提交请求放到一个异步任务队列中, 立即报告TC提交成功.
    * 异步任务队列的提交请求执行时只是删除相应的UNDO_LOG记录.
  * RM收到abort: 开启一个本地事务, 通过XID和Branch ID找到对应的UNDO_LOG记录, 用afterImage数据与当前数据比较.
    * 如果相同: 根据业务SQL和UNDO_LOG的beforeImage, 生成回滚SQL并执行. 提交本地事务, 并报告给TC.
    * 如果不同: 数据被全局事务之外的动作做了修改. 脏数据需手动处理, 根据日志历史修正数据或将对应的UNDO_LOG记录删除.

优缺点:
* 并发度高
* SQL限制: [link](https://seata.apache.org/zh-cn/docs/user/sqlreference/sql-restrictions)
* 事务隔离限制: [link](https://seata.apache.org/zh-cn/docs/user/appendix/isolation)


proxy: 
* `DataSourceProxy`: `getConnection(...)`
* `ConnectionProxy`: `prepareStatement(...)`
  * `commit()`
    * 处于`@GlobalTransaction`中: 注册分支事务, 获取全局锁; UNDO_LOG记录入库; 提交本次事务.
    * 处于`@GlocakLock`中: 查询TC是否有全局锁, **如果有则抛出异常**; 提交本次事务.
    * 其他: 提交本次事务.
* `StatementProxy`
  * `executeXXX()`
    * 在DML中: 查询前镜像(with SELECT FOR UPDATE, 获取本地锁), 执行业务SQL, 查询后镜像, 准备UNDO_LOG记录 
    * SELECT FOR UPDATE: 先执行SELECT FOR UPDATE(获取本地锁), 如果处于`@GlobalTransaction`或`@GlocakLock`中, 检查是否有全局锁. 
      * 如果有全局锁, 会尝试回滚本地事务释放本地锁, 再争抢本地锁和全局锁, 直到拿到全局锁.
  
事务隔离: 

* `@GlobalTransaction`: 标识一个全局事务
* `@GlocalLock` + SELECT FOR UPDATE: 先获取本地锁, 在查询该记录是否有全局锁, 如果有抛出LockConflictException.

TCC

SAGA: 状态机
1. 调用Seata Server开始全局事务: xid - 对应一个状态机
2. 在本地数据库中记录装胎器已启动.
3. 注册分支事务: branch xid - 对应一个状态a
4. 在本地数据库中记录状态a已启动.
5. 执行状态a中业务逻辑, 在本地数据库中记录状态a已完成, 
6. 同时报告Seata Server分支事务状态.
7. 最后一个状态z执行完后, 在本地数据库中记录状态机已完成
8. 向Seata Server提交/回滚全局事务.

状态a执行完成后, 产生路由消息放入EventQueue, 事件消费者取出消息后, 执行后续状态.

# Apache ShardingShpere
* https://shardingsphere.apache.org/

> The distributed SQL transaction & query engine for data sharding, scaling, encryption, and more - on any database.

- ShardingSphere-JDBC: a lightweight Java framework that provides additional services at Java’s JDBC layer.
- ShardingSphere-Proxy: a transparent database proxy, providing a database server that encapsulates database binary protocol to support heterogeneous languages.

features:
- data sharding
- distributed transaction
- read/write splitting
- data migration
- query federation
- data encryption
- shadow database: for full-link online load testing.

actions:
* workbench/Java/JavaEE/example-springcloud/persistent/shardingsphere
* rtfsc: rtfsc\shardingsphere